## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [2]:
path = "/Users/ecem/Desktop/phonocardiogram/data/AV/absent"

In [3]:
waves = np.load(path + "/absent_AV_waves_10sec.npy", allow_pickle= True)

In [4]:
waves.shape

(6980, 40000)

## comparison of two dataset:

Now we will to try visualize spectograms. So far we have used Obspy to visualize SCG signals. However, with obspy we are not able to plot spectograms side by side. This is important for us because we want to see spectograms of x,y and z components simultaneously for both healty and diseased people. So, from now on I will refer to those plots, for 3 components of healty and diseased (in total six plot, as "PAIR".


I have mentioned that Obpsy has a problem in side by side plotting. So I changed it source code. I have added and deleted some part to operate according to my need. ???????????

Thats why from now on we are going to use **output_spectogram** function to get values needed to plot spectogram.

These are the imports for output_spectogram function to work.

In [5]:
from obspy.imaging.cm import obspy_sequential
import math
from matplotlib import mlab
from matplotlib.colors import Normalize

In [6]:
def _nearest_pow_2(x):
    a = math.pow(2, math.ceil(np.log2(x)))
    b = math.pow(2, math.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b

In [7]:
def output_spectrogram(data, samp_rate = 4000 , per_lap=0.9, wlen=None, log=False,
                outfile=None, fmt=None, axes=None, dbscale=False,
                mult=8.0, cmap=obspy_sequential, zorder=None, title=None,
                show=True, clip=[0.0, 1.0]):
 
    import matplotlib.pyplot as plt
    
    # enforce float for samp_rate
    samp_rate = float(samp_rate)

    # set wlen from samp_rate if not specified otherwise
    if not wlen:
        wlen = samp_rate / 100.

    npts = len(data)
    # nfft needs to be an integer, otherwise a deprecation will be raised
    # XXX add condition for too many windows => calculation takes for ever
    nfft = int(_nearest_pow_2(wlen * samp_rate))
    if nfft > npts:
        nfft = int(_nearest_pow_2(npts / 8.0))

    if mult is not None:
        mult = int(_nearest_pow_2(mult))
        mult = mult * nfft
    nlap = int(nfft * float(per_lap))

    data = data - np.array(data).mean()
    end = npts / samp_rate

    # Here we call not plt.specgram as this already produces a plot
    # matplotlib.mlab.specgram should be faster as it computes only the
    # arrays
    # XXX mlab.specgram uses fft, would be better and faster use rfft
    specgram, freq, time = mlab.specgram(data, Fs=samp_rate, NFFT=nfft,
                                         pad_to=mult, noverlap=nlap, mode = 'psd')


    # db scale and remove zero/offset for amplitude
    if dbscale:
        specgram = 10 * np.log10(specgram[1:, :])
    else:
        specgram = np.sqrt(specgram[1:, :])
    freq = freq[1:]
    
    vmin, vmax = clip
    if vmin < 0 or vmax > 1 or vmin >= vmax:
        msg = "Invalid parameters for clip option."
        raise ValueError(msg)
    _range = float(specgram.max() - specgram.min())
    vmin = specgram.min() + vmin * _range
    vmax = specgram.min() + vmax * _range
    norm = Normalize(vmin, vmax, clip=True)

    # calculate half bin width
    halfbin_time = (time[1] - time[0]) / 2.0
    halfbin_freq = (freq[1] - freq[0]) / 2.0

    # argument None is not allowed for kwargs on matplotlib python 3.3
    kwargs = {k: v for k, v in (('cmap', cmap), ('zorder', zorder))
              if v is not None}

    if log:
        # pcolor expects one bin more at the right end
        freq = np.concatenate((freq, [freq[-1] + 2 * halfbin_freq]))
        time = np.concatenate((time, [time[-1] + 2 * halfbin_time]))
        # center bin
        time -= halfbin_time
        freq -= halfbin_freq
        # Log scaling for frequency values (y-axis)
        ax.set_yscale('log')
        # Plot times
        ax.pcolormesh(time, freq, specgram, norm=norm, **kwargs)
    else:
        # this method is much much faster!
        specgram = np.flipud(specgram)
        # center bin
        extent = (time[0] - halfbin_time, time[-1] + halfbin_time,
                  freq[0] - halfbin_freq, freq[-1] + halfbin_freq)
    return [specgram, extent, end]

    



In [8]:
output_spectrogram(waves[0].data, 4000, per_lap = 0.2)[0].shape

(16384, 11)

## Plotting spectogram:

f is the frequency array, containing the frequencies of every band of the fft. Which can be used as the labels for a graph

t is the time array, containing the time at which this FFT was made relative to the source signal. Again can be used for labels.

The Sxx array contains the amplitudes and is a 2d array whose shape is the length of f by the length of t.


 The purpose of a spectogram is to take the FFT of small, equal-sized time chunks. This produces a 2D fourier transform where the X axis is the start time of the time chunk and the Y axis is the energy (or power, etc.) in each frequency in that time chunk. This allows you to see how the frequency components change over time.
 
  A spectrogram is a representation of frequency over time with the addition of amplitude as a third dimension, denoting the intensity or volume of the signal at a frequency and a time.

In [9]:
def freq_average(array):
    means= []
    for i in range(array.shape[0]):
        means.append(array[i][0:88].mean())
    return np.array(means)
        

In [10]:
def spec_values(array):
    x_ = []
    for i in range(array.shape[0]):
        _x = output_spectrogram(array[i].data, 4000, per_lap = 0.2)[0]

        x_.append(freq_average(array = _x))
        
        
    return np.array(x_)

In [11]:
waves_ = spec_values(waves)

In [12]:
waves_[0]

array([9.03286628e-11, 1.28330686e-10, 1.27376206e-10, ...,
       9.23179423e-04, 9.16569553e-04, 9.08058914e-04])

In [13]:
print(waves_.shape)


(6980, 16384)


In [14]:
columns = []
for i in range(waves_.shape[1]):
    columns.append("freq_{}".format(i))
print(columns)

['freq_0', 'freq_1', 'freq_2', 'freq_3', 'freq_4', 'freq_5', 'freq_6', 'freq_7', 'freq_8', 'freq_9', 'freq_10', 'freq_11', 'freq_12', 'freq_13', 'freq_14', 'freq_15', 'freq_16', 'freq_17', 'freq_18', 'freq_19', 'freq_20', 'freq_21', 'freq_22', 'freq_23', 'freq_24', 'freq_25', 'freq_26', 'freq_27', 'freq_28', 'freq_29', 'freq_30', 'freq_31', 'freq_32', 'freq_33', 'freq_34', 'freq_35', 'freq_36', 'freq_37', 'freq_38', 'freq_39', 'freq_40', 'freq_41', 'freq_42', 'freq_43', 'freq_44', 'freq_45', 'freq_46', 'freq_47', 'freq_48', 'freq_49', 'freq_50', 'freq_51', 'freq_52', 'freq_53', 'freq_54', 'freq_55', 'freq_56', 'freq_57', 'freq_58', 'freq_59', 'freq_60', 'freq_61', 'freq_62', 'freq_63', 'freq_64', 'freq_65', 'freq_66', 'freq_67', 'freq_68', 'freq_69', 'freq_70', 'freq_71', 'freq_72', 'freq_73', 'freq_74', 'freq_75', 'freq_76', 'freq_77', 'freq_78', 'freq_79', 'freq_80', 'freq_81', 'freq_82', 'freq_83', 'freq_84', 'freq_85', 'freq_86', 'freq_87', 'freq_88', 'freq_89', 'freq_90', 'freq_91

In [15]:
print(len(columns))

16384


In [16]:
def dataframe_creator(array, df = pd.DataFrame()):
    values = []
    for i in range(array.shape[0]):
        values.append(array[i][0:16384].tolist())

        
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [17]:
#df = dataframe_creator(waves_, 
                         #df =pd.DataFrame(columns = columns))

In [18]:
#df.to_csv(dir_path + '/spectogram_coef_AV.csv')

In [19]:
np.save(path + "/absent_AV_spectogram_coef", waves_, allow_pickle= True)